In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Define threshold for collinearity check
collinearity_threshold = 0.05  # Adjust as needed

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        # Convert the image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Detect the pose
        results = pose.process(image)
        
        # Convert the image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Check if pose landmarks are detected
        if results.pose_landmarks:
            # Extract landmark coordinates
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates of relevant landmarks
            left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
            right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
            left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y])
            right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y])
            
            # Calculate vectors representing shoulders and hips
            vec_shoulder = right_shoulder - left_shoulder
            vec_hip = right_hip - left_hip
            
            # Calculate cross product to check collinearity
            cross_product = np.cross(vec_shoulder, vec_hip)
            
            # Check if the cross product magnitude is below the threshold
            posture_correct = np.linalg.norm(cross_product) < collinearity_threshold
            
            # Visualize the detected posture on the screen
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Display posture evaluation result
            posture_text = "Correct" if posture_correct else "Incorrect"
            cv2.putText(image, f"Posture: {posture_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if posture_correct else (0, 0, 255), 2)
            
        cv2.imshow('Posture Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


AttributeError: SPINE_MID

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Define thresholds for bad posture conditions
angle_threshold = 20  # Maximum allowed angle between shoulders and hips (in degrees)
curve_threshold = 0.05  # Maximum allowed deviation from a straight spine
shoulder_threshold = 0.1  # Maximum allowed deviation in shoulder position (in meters)
hip_threshold = 0.1  # Maximum allowed deviation in hip position (in meters)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        # Convert the image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Detect the pose
        results = pose.process(image)
        
        # Convert the image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Check if pose landmarks are detected
        if results.pose_landmarks:
            # Extract landmark coordinates
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates of relevant landmarks
            left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
            right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
            left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y])
            right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y])
            
            # Calculate midpoint between shoulders (approximation for spine)
            spine = (left_shoulder + right_shoulder) / 2
            
            # Calculate vectors representing shoulders and hips
            vec_shoulder = right_shoulder - left_shoulder
            vec_hip = right_hip - left_hip
            
            # Calculate angle between shoulders and hips
            angle_shoulder_hip = math.degrees(math.atan2(np.linalg.norm(np.cross(vec_shoulder, vec_hip)), np.dot(vec_shoulder, vec_hip)))
            
            # Check if the angle exceeds the threshold
            posture_correct = angle_shoulder_hip < angle_threshold
            
            # Check if the spine is significantly curved
            spine_curve = abs(np.cross(left_shoulder - spine, right_shoulder - spine))
            posture_correct = posture_correct and (spine_curve < curve_threshold)
            
            # Check if shoulders are hunched forward or slouched
            shoulder_deviation = max(np.linalg.norm(left_shoulder - right_shoulder), np.linalg.norm(left_shoulder - spine), np.linalg.norm(right_shoulder - spine))
            posture_correct = posture_correct and (shoulder_deviation < shoulder_threshold)
            
            # Check if hips are tilted or uneven
            hip_deviation = max(np.linalg.norm(left_hip - right_hip), np.linalg.norm(left_hip - spine), np.linalg.norm(right_hip - spine))
            posture_correct = posture_correct and (hip_deviation < hip_threshold)
            
            # Visualize the detected posture on the screen
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Display posture evaluation result
            posture_text = "Correct" if posture_correct else "Incorrect"
            cv2.putText(image, f"Posture: {posture_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if posture_correct else (0, 0, 255), 2)
            
        cv2.imshow('Posture Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [21]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 90:
        angle = 180 - angle

    return angle

# Function to check if the person is standing straight
def is_straight(landmarks):
    # Define the key points for the upper body
    shoulders = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    hips = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value], landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

    # Calculate the angles
    angle1 = calculate_angle(shoulders[0], hips[0], knee[0])  # Left side
    angle2 = calculate_angle(shoulders[1], hips[1], knee[1])  # Right side
    print(angle1, angle2)

    # If both angles are less than a threshold, the person is standing straight
    return angle1 < 6 and abs(angle2) < 10

# Main function
def main():
    cap = cv2.VideoCapture(0)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Convert the image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Convert the image back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
                is_standing_straight = is_straight(landmarks)

                # Draw landmarks on the image
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Display status
                if is_standing_straight:
                    cv2.putText(image, "Standing straight", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                else:
                    cv2.putText(image, "Not standing straight", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.imshow('MediaPipe Pose', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


5.4352576853098356 -3.081225096682772
3.8853567696891673 -4.4263757420652325
3.265532340886807 -5.056967358416017
3.1991534749297728 -5.164893247315916
2.8600018389452373 -5.552300960686551
2.7295826663646494 -5.759881964129988
2.3785293335983795 -6.253303679484162
2.2306022051689354 -6.520166067554925
2.328513045168137 -6.57580874782434
2.3913009426584324 -6.5881463981989725
2.450828909576842 -6.393103479031964
2.4943816449406597 -6.274619629019838
2.5428729821457807 -6.1885438591350805
2.574022719214099 -6.10798218126169
2.5940808851971724 -6.060425379204787
2.6368796407979573 -5.859143358562591
2.6646445574159827 -5.746889857449389
2.6949801849729624 -5.670077629290574
2.740198773998202 -5.587918117790025
2.779430034311588 -5.526798260844856
2.737398229313328 -5.552009527222793
2.7093327078177936 -5.571302195920168
2.7508966644314228 -5.495711446336259
2.7819560875869342 -5.43658771819392
2.826005299262988 -5.43875345539999
2.9979828740227106 -5.215308443178998
3.0790190403916995 -5